In [1]:
import requests
import pandas as pd
import urllib
import json

In [2]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 16D9-A643

 Directory of C:\Users\dsilva2\EMAT22110_Office

10/13/2021  09:21 AM    <DIR>          .
10/13/2021  09:21 AM    <DIR>          ..
10/13/2021  08:59 AM    <DIR>          .ipynb_checkpoints
09/21/2021  01:20 PM           313,653 DataFrame-Methods-and-visuals.ipynb
09/20/2021  01:46 PM           134,652 emoji-encodings-from-fb-data.ipynb
09/07/2021  02:14 PM    <DIR>          facebook-davidsilva100046_full
09/07/2021  02:10 PM     1,857,380,869 facebook-davidsilva100046_full.zip
09/07/2021  02:14 PM    <DIR>          facebook-davidsilva100046_year
09/07/2021  02:09 PM       475,994,340 facebook-davidsilva100046_year.zip
09/06/2021  10:05 AM             1,253 image_resize.ipynb
09/28/2021  10:52 AM               102 likes_report_grades-anony.csv
09/17/2021  11:41 AM            19,805 likes_Report_playing.ipynb
09/28/2021  12:08 PM            89,872 likes-Report_Grade_Analysis-FA21.ipynb
09/15/2021  10:50 AM             2,6

In [3]:
bearer_token = pd.read_csv('Twitter-Keys_10-5-21.txt', sep = '\t', header = 0)

In [4]:
# bearer_token = pd.read_csv('Twitter-Keys_10-5-21.txt', sep = '\t', header = 0).T
# bearer_token.columns = bearer_token.iloc[0]
# bearer_token = bearer_token[1:]

In [5]:
# bearer_token['Bearer_Token'].iloc[0]

### I had `Authentication` in a previous example when it should be `Authorization`.

In [6]:
header = {'Authorization' : 'Bearer {}'.format(bearer_token['Bearer_Token'].iloc[0])}

In [7]:
bearer_token.keys()

Index(['Bearer_Token'], dtype='object')

`response = requests.request("GET", url, headers = header)`

In [8]:
endpoint_url = 'https://api.twitter.com/2/tweets/search/recent'

`#Crew96 lang:en`

`(#Crew96 OR @ColumbusCrew) lang:en`

In [9]:
query = urllib.parse.quote('(#Crew96 OR @ColumbusCrew) lang:en')

In [10]:
query

'%28%23Crew96%20OR%20%40ColumbusCrew%29%20lang%3Aen'

In [11]:
simple_api_url = endpoint_url + '?query={}'.format(query)

In [12]:
simple_api_url

'https://api.twitter.com/2/tweets/search/recent?query=%28%23Crew96%20OR%20%40ColumbusCrew%29%20lang%3Aen'

If you want to specify parameters other than `query`, you will need to develop a slightly more detailed URL.<br>
Here we are setting a few `tweet_fields` to add to the URL.<br>
`public_metrics,created_at,author_id,lang`

In [13]:
tweet_fields = 'public_metrics,created_at,author_id,lang'

In [14]:
slightly_more_complex_api_url = endpoint_url + '?query={}&tweet.fields={}'.format(query, tweet_fields)

In [15]:
slightly_more_complex_api_url

'https://api.twitter.com/2/tweets/search/recent?query=%28%23Crew96%20OR%20%40ColumbusCrew%29%20lang%3Aen&tweet.fields=public_metrics,created_at,author_id,lang'

Depending on the response fields you want from Twitter, you may need to specify `expansions`. Not all the data for a user, their geographic location, prior interactions, etc. are stored at the tweet level. Adding one (or several) expansion will let you collect those other response fields. In the developer documentation, the response field definitions specify which fields require which expansion.<br>
Here we are just going to grab the `username` field under the `author_id` expansion. The documentation specifies that `username` should be passed to the `user.fields` parameter instead of `tweet.fields` so we will do that also.

In [16]:
expansions = 'author_id'

Also notice that I have set `max_results=10` which will limit how many tweets I receive for each request. If left unspecified, the default is 10 for the search/recent endpoint. It can be set up to 100.

In [17]:
expansions_api_url = endpoint_url + '?query={}&max_results=10&tweet.fields={}&expansions={}&user.fields={}'.format(query, tweet_fields, expansions, 'username')

In [18]:
expansions_api_url

'https://api.twitter.com/2/tweets/search/recent?query=%28%23Crew96%20OR%20%40ColumbusCrew%29%20lang%3Aen&max_results=10&tweet.fields=public_metrics,created_at,author_id,lang&expansions=author_id&user.fields=username'

Now we are ready to make our first request.

In [19]:
response_1 = requests.request("GET", expansions_api_url, headers = header)

In [20]:
response_1.text

'{"data":[{"public_metrics":{"retweet_count":0,"reply_count":0,"like_count":3,"quote_count":0},"text":"@USMNT @VW @LowerFieldCbus @ColumbusCrew @tyler_adams14 Lose or draw, fire Gregg as he walks off the pitch.","author_id":"1370726106731401220","lang":"en","created_at":"2021-10-13T13:19:58.000Z","id":"1448277333874393099"},{"public_metrics":{"retweet_count":0,"reply_count":0,"like_count":0,"quote_count":0},"text":"@USMNT @VW @LowerFieldCbus @ColumbusCrew @tyler_adams14 Come back with your shields, or on them.","author_id":"3128076642","lang":"en","created_at":"2021-10-13T13:19:06.000Z","id":"1448277113807642635"},{"public_metrics":{"retweet_count":0,"reply_count":0,"like_count":0,"quote_count":0},"text":"@USMNT @VW @LowerFieldCbus @ColumbusCrew @tyler_adams14 Please play a competent lineup. I can confidently say that the Panama was the least entertaining game I’ve ever watched.","author_id":"1413634280752881664","lang":"en","created_at":"2021-10-13T13:17:19.000Z","id":"144827666722348

In [21]:
response_1_dict = json.loads(response_1.text)

In [22]:
response_1_dict.keys()

dict_keys(['data', 'includes', 'meta'])

Notice that the dictionary has `data` which is where the `tweet.fields` data is stored. This response also has an `includes` key, which is where the `author_id` and `username` expansions data is located. Finally is `meta` which is where we will find the `next_token` for pagination.

In [23]:
my_df = pd.DataFrame(response_1_dict['data'])

In [24]:
my_df.head(5)

,public_metrics,text,author_id,lang,created_at,id
0,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@USMNT @VW @LowerFieldCbus @ColumbusCrew @tyle...,1370726106731401220,en,2021-10-13T13:19:58.000Z,1448277333874393099
1,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@USMNT @VW @LowerFieldCbus @ColumbusCrew @tyle...,3128076642,en,2021-10-13T13:19:06.000Z,1448277113807642635
2,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@USMNT @VW @LowerFieldCbus @ColumbusCrew @tyle...,1413634280752881664,en,2021-10-13T13:17:19.000Z,1448276667223486467
3,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@USMNT @VW @LowerFieldCbus @ColumbusCrew @tyle...,3376699534,en,2021-10-13T13:17:09.000Z,1448276623489323009
4,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@USMNT @VW @LowerFieldCbus @ColumbusCrew @tyle...,495314632,en,2021-10-13T13:15:33.000Z,1448276223289868294


We can save this DataFrame just like any other DataFrame.

In [25]:
# my_df.to_csv("twitter_data.csv")

In [26]:
response_1_dict['meta']

{'newest_id': '1448277333874393099',
 'oldest_id': '1448275863229894657',
 'result_count': 10,
 'next_token': 'b26v89c19zqg8o3fpds9dr28xsh0sikrwlfde10b18u4d'}

Now we can add the `next_token` to the URL to specify the next page of data.

In [27]:
expansions_api_url_2 = expansions_api_url + '&next_token={}'.format(response_1_dict['meta']['next_token'])

In [28]:
expansions_api_url_2

'https://api.twitter.com/2/tweets/search/recent?query=%28%23Crew96%20OR%20%40ColumbusCrew%29%20lang%3Aen&max_results=10&tweet.fields=public_metrics,created_at,author_id,lang&expansions=author_id&user.fields=username&next_token=b26v89c19zqg8o3fpds9dr28xsh0sikrwlfde10b18u4d'

In [29]:
response_2 = requests.request("GET", expansions_api_url_2, headers = header)

In [30]:
response_2.text

'{"data":[{"public_metrics":{"retweet_count":0,"reply_count":0,"like_count":0,"quote_count":0},"text":"@USMNT @VW @LowerFieldCbus @ColumbusCrew @tyler_adams14 Hopefully we get a shot on goal tonight \\uD83D\\uDE00","created_at":"2021-10-13T13:13:50.000Z","author_id":"1416160979558146050","lang":"en","id":"1448275791289126919"},{"public_metrics":{"retweet_count":20,"reply_count":0,"like_count":0,"quote_count":0},"text":"RT @ColumbusCrew: “It’s almost like a dream come true.”\\n\\nSpecial bond between Gregg Berhalter &amp; Columbus.\\n\\n#Crew96 | #USMNT https://t.co/Fq…","created_at":"2021-10-13T13:13:27.000Z","author_id":"350904219","lang":"en","id":"1448275692895019018"},{"public_metrics":{"retweet_count":0,"reply_count":0,"like_count":8,"quote_count":0},"text":"@USMNT @VW @LowerFieldCbus @ColumbusCrew @tyler_adams14 I hope that chapter isn’t one that is in a horror novel…","created_at":"2021-10-13T13:12:43.000Z","author_id":"1079235200268275712","lang":"en","id":"1448275510207950851"

We can `append` this new data to the `my_df` DataFrame, which will add the new rows to that data. I'm going to save a step and not create a new dictionary for the second response and instead do it all in one line.<br>
The `ignore_index` parameter being set to `True` will overwrite the index for the new rows. Otherwise the DataFrame would have indices 0,1,2...,8,9,0,1,2...,8,9.

In [31]:
my_df = my_df.append(pd.DataFrame(json.loads(response_2.text)['data']), ignore_index= True)

In [32]:
my_df.tail(3)

,public_metrics,text,author_id,lang,created_at,id
17,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",RT @783nate: What an amazing moment! #Crew96 #...,710323992360517632,en,2021-10-13T12:57:02.000Z,1448271562986201095
18,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Happy Birthday to the Best Prez @tbez10 @Colum...,221496920,en,2021-10-13T12:54:05.000Z,1448270820137177089
19,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Morning all...\n\nGiven the #usmnt are playing...,322965853,en,2021-10-13T12:44:39.000Z,1448268447092289536


Below is the start to a function that will complete the pagination up to a set number of pages. It's not a complete function and there are many ways to code the same process. This is not required for the upcoming assignment.

In [33]:
def twitter_api_pagination(num_pages, my_url):
    count = 0
    next_token = None
    base_url = my_url
    full_data = pd.DataFrame()
    while count < num_pages:
        if next_token == None:
            first_response = requests.request("GET", base_url, headers = header)
            # if raise(Exception)....
            response = json.loads(first_response)
            full_data = full_data.append(pd.DataFrame(response['data']))
        else:
            url_new = base_url + '&next_token={}'.format(response['meta']['next_token'])
            responses = first_response = requests.request("GET", url_new, headers = header)
            # if raise(Exception)....
            response = json.loads(first_response)
            full_data = full_data.append(pd.DataFrame(response['data']))
    count += 1
    return full_data

After defining the function, the output can be saved as a variable.<br>
This currently does not run correctly!

In [ ]:
my_new_df = twitter_api_pagination(3, expansions_api_url)